In [ ]:
!pip install --upgrade langchain deeplake openai tiktoken

In [ ]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ['OPENAI_API_KEY'] = ''
embeddigns = OpenAIEmbeddings()

In [ ]:
!activeloop login -t -----------

In [ ]:
import os
from langchain.document_loaders import TextLoader

root_dir = './the-algorithm-main'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
  for file in filenames:
    try:
      loader = TextLoader(os.path.join(dirpath, file), encoding = 'utf-8')
      docs.extend(loader.load_and_split())
    except Exception as e:
      pass

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlay=0)
texts = text_splitter.split_documents(docs)

In [ ]:
db = DeepLake.from_documents(texts, embeddings, dataset_path = "hub://mg/twitter-algorithm")

In [ ]:
db = DeepLake(dataset_path = "hub://mg/twitter-algorithm", read_only=True, embedding_function = embeddings)

In [ ]:
retreiver = db.as_retreiver()
retreiver.search_kwargs['distance_metric'] = 'cos'
retreiver.search_kwargs['fetch_k'] = 100
retreiver.search_kwargs['maximal_marginal_relevance'] = True
retreiver.search_kwargs['k'] = 20


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model='gpt-3.5-turbo')
qa = ConversationalRetrievalChain.from_llm(model, retriever = retriever)

In [ ]:
questions = [
    'What does setup.py code do ?'
]

chat_history=[]

for q in questions:
  result = qa({"question":q, "chat_history":chat_history})
  chat_history.append((q, result['answer']))
  print(f"-> **Question**:{q} \n")
  print(f"**Answer**: {result['answer']} \n")